In [24]:
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import keras_applications, keras_preprocessing

In [36]:
data_path = os.path.join(os.getcwd(), 'global-wheat-detection', 'train' + '.' + 'csv')

In [41]:
df_train = pd.DataFrame(pd.read_csv(data_path))

In [42]:
print(df_train.shape)
df_train.head()


(147793, 5)


,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1
